In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
print("Libraries loaded successfully")

Libraries loaded successfully


In [2]:
IMAGE_SIZE = [224, 224]

train_path = 'Datasets/Train'
valid_path = 'Datasets/Test'

In [3]:
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [4]:
for layer in resnet.layers:
    layer.trainable = False

In [5]:
folders = glob('Datasets/Train/*')
folders

['Datasets/Train\\Ford Mustang',
 'Datasets/Train\\Hummer',
 'Datasets/Train\\McLaren',
 'Datasets/Train\\Mercedes Benz',
 'Datasets/Train\\Others']

In [6]:
x = Flatten()(resnet.output)

In [7]:
prediction = Dense(len(folders), activation='softmax')(x)

model = Model(inputs=resnet.input, outputs=prediction)

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [9]:
model.compile(
  loss='categorical_crossentropy',
  optimizer= 'adam',
  metrics=['accuracy']
)

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [11]:
training_set = train_datagen.flow_from_directory('Datasets/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

Found 506 images belonging to 5 classes.


In [12]:
training_set.class_indices

{'Ford Mustang': 0, 'Hummer': 1, 'McLaren': 2, 'Mercedes Benz': 3, 'Others': 4}

In [13]:
test_set = test_datagen.flow_from_directory('Datasets/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 46 images belonging to 5 classes.


In [14]:
test_set.class_indices

{'Ford Mustang': 0,
 'Hummer': 1,
 'McLaren': 2,
 'Mercerdes Benz': 3,
 'Others': 4}

In [15]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=100,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

C:\Users\anand\AppData\Local\Temp\ipykernel_10308\985405674.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/100
32/32 [==============================] - 34s 994ms/step - loss: 5.2780 - accuracy: 0.2609 - val_loss: 3.8472 - val_accuracy: 0.3478
Epoch 2/100
32/32 [==============================] - 30s 931ms/step - loss: 2.1057 - accuracy: 0.4012 - val_loss: 2.2763 - val_accuracy: 0.3913
Epoch 3/100
32/32 [==============================] - 30s 921ms/step - loss: 1.5837 - accuracy: 0.4486 - val_loss: 1.7278 - val_accuracy: 0.3043
Epoch 4/100
32/32 [==============================] - 29s 906ms/step - loss: 1.9570 - accuracy: 0.4783 - val_loss: 2.1829 - val_accuracy: 0.3913
Epoch 5/100
32/32 [==============================] - 29s 895ms/step - loss: 1.6340 - accuracy: 0.5336 - val_loss: 1.5562 - val_accuracy: 0.4130
Epoch 6/100
32/32 [==============================] - 30s 947ms/step - loss: 1.3439 - accuracy: 0.5652 - val_loss: 2.1844 - val_accuracy: 0.4565
Epoch 7/100
32/32 [==============================] - 35s 1s/step - loss: 2.0251 - accuracy: 0.4644 - val_loss: 1.7032 - val_accuracy: 0.

In [16]:
from tensorflow.keras.models import load_model

model.save('model_resnet50.h5')

In [19]:
load_model("model_resnet50.h5")